<a href="https://colab.research.google.com/github/kirkwilson/annotate_images/blob/main/annotate_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**upload files**
*   needs to be png for jpeg
*   can be a zip of images

**write text**
*   variable for text to write
*   standard position on image (bottom left corner based on percentage)

**export files**
*   export image
*   zip if multiple files

In [5]:
TEXT = 'Lorem ipsum...' #@param {type:"string"}
BUFFER_PERCENT = 0.05 #@param {type:"number"}
TEXT_HEIGHT_PERCENT = 0.05 #@param {type:"number"}

In [6]:
import os.path
import zipfile
from google.colab import files
from PIL import Image, ImageFont, ImageDraw

In [7]:
from re import T
DIRECTORY = '/content/decompressed_files/'
FONT_LOCATION = 'LiberationSerif-Regular.ttf'

def upload_files():
  dict_files = files.upload()
  return dict_files

def decompress_file(file_name):
  with zipfile.ZipFile(file_name, 'r') as zip_ref:
    zip_ref.extractall(DIRECTORY)

def decompress_files(files):
  for file_name in files.keys():
    decompress_file(file_name)

def upload_and_decompress():
  files = upload_files()
  decompress_files(files)

def writable_area(image, buffer):
  left = image.width * buffer
  top = image.height * buffer
  right = image.width - (image.width * buffer)
  bottom = image.height - (image.height * buffer)
  return(left, top, right, bottom)
  
def text_anchor(bbox):
  x = bbox[0]
  y = bbox[1]
  return (x, y)

def stroke_size(font):
  pixel_height = bbox_dimensions(font)[1]
  stroke_width = int(pixel_height*0.1)
  if stroke_width == 0:
    stroke_width = 1
  return stroke_width

def create_font(font_location, font_size=12):
  font = ImageFont.truetype(font_location, font_size)
  return font

def font_text_bbox(font, text):
  return font.getbbox(text)

def bbox_dimensions(bbox):
  width = bbox[2] - bbox[0]
  height = bbox[3] - bbox[1]
  return (width, height)

def text_bbox(bbox, text_height):
  height = bbox_dimensions(bbox)[1]
  left = bbox[0]
  top = height - (height * text_height)
  right = bbox[2]
  bottom = bbox[3]
  return(left, top, right, bottom)

def writable_text_area(image, buffer, text_height):
  writable_bbox = writable_area(image, buffer)
  writable_text_area = text_bbox(writable_bbox, text_height)
  return writable_text_area

def dimension_a_minus_b(a, b):
  a = bbox_dimensions(a)
  b = bbox_dimensions(b)
  result = tuple(map(lambda i, j: i - j, a, b))
  return result

def check_if_fit(difference_tuple):
  can_fit = True
  for difference in difference_tuple:
    if difference > 0:
      can_fit = False
  return can_fit

def can_a_fit_in_b(a_bbox, b_bbox):
  difference_tuple = dimension_a_minus_b(a_bbox, b_bbox)
  can_fit = check_if_fit(difference_tuple)
  return can_fit

def annotated_file_name(file_name):
  name, extension = file_name.split('.')
  annotated = name + '_annotated.' + extension
  return annotated

def scale_font(font_location, text, text_area):
  font = create_font(font_location)
  font_area = font_text_bbox(font, text)

  # edited code from here: https://stackoverflow.com/a/61891053
  jumpsize = 75
  fontsize = 1
  while True:
    if can_a_fit_in_b(font_area, text_area):
      fontsize += jumpsize
    else:
      jumpsize = int(jumpsize / 2)
      fontsize -= jumpsize
    font = create_font(FONT_LOCATION, fontsize)
    font_area = font_text_bbox(font, text)
    if jumpsize <= 1:
      break
  return font

# def walk_directory():
#   for root, dirs, files in os.walk(DIRECTORY):
#     for file in files:

In [8]:
file_name = 'example-img.png'

img = Image.open(file_name)

text_area = writable_text_area(img, BUFFER_PERCENT, TEXT_HEIGHT_PERCENT)
anchor = text_anchor(text_area)

draw = ImageDraw.Draw(img)

print(scale_font(FONT_LOCATION, TEXT, text_area))

# if can_a_fit_in_b(font_area, text_area):
#   stroke_width = stroke_size(font_area)
#   draw.text(anchor, TEXT, 
#             font=font, stroke_width=stroke_width, stroke_fill='black')
#   img.save(annotated_file_name(file_name))


# todo:
# walk directory
# iterate files